CONFIDENTIEL

Installation des dépendances

In [108]:
%pip install azure-storage-blob
%pip install pyyaml
%pip install pydub
%pip install praat-parselmouth
%pip install azure-cognitiveservices-speech


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [135]:
# Replace with your local path, for example:
# /Users/elodieroyant/Documents/StudioVRAI
LOCAL_PATH = input()

In [11]:
# import env variables (you should retrieve env in secure notes and save as a .env.yaml file at root file)
import yaml
with open('.env.yaml', 'r') as stream:
    data = yaml.safe_load(stream).split(':')
env = { data[0]: data[1] }

In [144]:
import json
import csv

def format_labels(json_data):
    id = json_data['id']
    file_name = json_data['task']['data']['video_url'].split('/')[-1]
    results_with_none = list(map(lambda x: x['value'] if x['type'] == 'rating' else None, json_data['result']))
    results = list(filter(lambda x: x is not None, results_with_none))
    rows = list(map(lambda x: { "id": id, "file_name": file_name, "rating": x["rating"], "start": x["start"],"end": x["end"], "duration": x["end"] - x["start"] }, results))
    return rows

headers = ["id", "file_name", "rating", "start", "end",  "duration"]

with open(LOCAL_PATH + f"/formated-labels/tchatche.csv", "w", newline='') as outfile:
    w = csv.DictWriter(outfile, headers)
    w.writeheader()
    for i in range(8, 131):
        # read dw json files and write them to a csv
        with open(LOCAL_PATH + f"/raw-labels/{i}.json", "r") as file:
            json_data = json.load(file)
            rows = format_labels(json_data)
            for row in rows:
                w.writerow(row)

with open(LOCAL_PATH + f"/formated-labels/tchatche.csv", newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    files = []
    for row in reader:
        if (row['file_name'] not in files):
            files.append(row['file_name'])

In [106]:
# Download a file from blob storage on local path
from azure.storage.blob import BlobServiceClient

# Create a BlobServiceClient object using your connection string
blob_service_client = BlobServiceClient.from_connection_string(env['AZURE_BLOB_STORAGE_CONNECTION_STRING'])
# Specify the container name and blob name
container_name = "raw"

for blob_name in files:
    # Get a reference to the blob
    blob_client = blob_service_client.get_blob_client(container_name, blob_name)

    # Download the blob content
    downloaded_content = blob_client.download_blob().readall()

    # Save the downloaded content to a file
    with open(LOCAL_PATH + f"/raw-data/{blob_name}", "wb") as file:
        file.write(downloaded_content)

In [ ]:
# extract audio from video
import subprocess

with open(LOCAL_PATH + f"/formated-labels/tchatche.csv", newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    files = []
    for row in reader:
        file_name = row['file_name'].replace('.mp4', '')
        if (file_name not in files):
            files.append(file_name)

for file in files:
    print(file)
    command = f"ffmpeg -i {LOCAL_PATH}/raw-data/{file}.mp4 -ab 160k -ac 2 -ar 44100 -vn {LOCAL_PATH}/raw-audio/{file}.wav"
    subprocess.call(command, shell=True)

In [ ]:
# file segmentation on labels
from pydub import AudioSegment

with open(LOCAL_PATH + f"/formated-labels/tchatche.csv", newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    files = []
    for row in reader:
        file_name = row['file_name'].replace('.mp4', '')
        start = int(float(row['start']) * 1000)
        end = int(float(row['end']) * 1000)
        path = f'{LOCAL_PATH}/raw-audio/{file_name}.wav'
        print(path)
        newAudio = AudioSegment.from_wav(path)
        newAudio = newAudio[start:end]
        newAudio.export(f'{LOCAL_PATH}/split-audio/{file_name}_{start}_{end}.wav', format="wav") #Exports to a wav file in the current path.


In [151]:
# new csv with correct file name
from pydub import AudioSegment

headers = ["id", "file_name", "rating", "duration"]

def parse_tsp_in_s(tps_in_s):
    return int(float(tps_in_s*1000))

def format_labels(json_data):
    file_name = json_data['task']['data']['video_url'].split('/')[-1].replace('.mp4', '')
    results_with_none = list(map(lambda x: x['value'] if x['type'] == 'rating' else None, json_data['result']))
    results = list(filter(lambda x: x is not None, results_with_none))
    rows = list(map(lambda x: { "file_name": f'{file_name}_{parse_tsp_in_s(x["start"])}_{parse_tsp_in_s(x["end"])}.wav', "rating": x["rating"], "duration": x["end"] - x["start"] }, results))
    return rows

headers = ["file_name", "rating", "duration"]

with open(LOCAL_PATH + f"/formated-labels/tchatche_split.csv", "w", newline='') as outfile:
    w = csv.DictWriter(outfile, headers)
    w.writeheader()
    for i in range(8, 131):
        # read dw json files and write them to a csv
        with open(LOCAL_PATH + f"/raw-labels/{i}.json", "r") as file:
            json_data = json.load(file)
            rows = format_labels(json_data)
            for row in rows:
                w.writerow(row)

with open(LOCAL_PATH + f"/formated-labels/tchatche_split.csv", newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    files = []
    for row in reader:
        if (row['file_name'] not in files):
            files.append(row['file_name'])

In [ ]:
import parselmouth
import numpy as np
import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(subscription=env['AZURE_SPEECH_KEY'], region=env['AZURE_SPEECH_REGION'], speech_recognition_language="fr-FR")

headers = ["file_name", "rating", "duration", "transcript", "speed",\
           "intensity_avg", "intensity_std", "intensity_min", "intensity_max", \
            "pitch_avg", "pitch_std", "pitch_min", "pitch_max",
            "mfcc0_avg", "mfcc0_std", "mfcc0_min", "mfcc0_max",
            "mfcc1_avg", "mfcc1_std", "mfcc1_min", "mfcc1_max",
            "mfcc2_avg", "mfcc2_std", "mfcc2_min", "mfcc2_max"]

new_rows = []
with open(LOCAL_PATH + f"/data-csv/tchatche_split.csv", "r", newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        new_rows.append(row)

row = new_rows[0]
snd = parselmouth.Sound(LOCAL_PATH + f"/raw-audio/{row['file_name']}")
# intensity = snd.to_intensity()
# print(intensity.values)
# print(intensity.values[0])
# pitch = snd.to_pitch().selected_array['frequency']
# print(pitch)
# pitch = pitch[pitch != 0]
# print(pitch)
# mfcc_object = snd.to_mfcc(number_of_coefficients=12)
# mfcc = mfcc_object.to_array()
# print(mfcc_object)
# print(mfcc.shape)
# for i in range(0,12): 
#  print('avg', np.mean(mfcc[i]))
#  print('std', np.std(mfcc[i]))
#  print('min', np.min(mfcc[i]))
#  print('max', np.max(mfcc[i]))
# audio_config = speechsdk.audio.AudioConfig(filename=LOCAL_PATH + f"/raw-audio/{row['file_name']}")
# speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
# result = speech_recognizer.recognize_once()
# transcript = result.text
# speed = len(transcript) * 60 / float(row['duration'])
# print(speed)


with open(LOCAL_PATH + f"/data-csv/tchatche_split_x.csv", "w", newline='') as csv_file:
    w = csv.DictWriter(csv_file, headers)
    w.writeheader()
    for row in new_rows:
        audio_config = speechsdk.audio.AudioConfig(filename=LOCAL_PATH + f"/raw-audio/{row['file_name']}")
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
        result = speech_recognizer.recognize_once()
        row['transcript'] = result.text
        row['speed'] = len(row['transcript'].split(" ")) * 60 / float(row['duration'])
        snd = parselmouth.Sound(LOCAL_PATH + f"/raw-audio/{row['file_name']}")
        intensity = snd.to_intensity()
        row['intensity_avg'] = np.mean(intensity.values[0])
        row['intensity_std'] = np.std(intensity.values[0])
        row['intensity_min'] = np.min(intensity.values[0])
        row['intensity_max'] = np.max(intensity.values[0])
        pitch = snd.to_pitch().selected_array['frequency']
        pitch = pitch[pitch != 0]
        row['pitch_avg'] = np.mean(pitch)
        row['pitch_std'] = np.std(pitch)
        row['pitch_min'] = np.min(pitch)
        row['pitch_max'] = np.max(pitch)
        mfcc_object = snd.to_mfcc(number_of_coefficients=12)
        mfcc = mfcc_object.to_array()
        for i in range(0,3): 
            row[f'mfcc{i}_avg'] = np.mean(mfcc[i])
            row[f'mfcc{i}_std'] = np.std(mfcc[i])
            row[f'mfcc{i}_min'] = np.min(mfcc[i])
            row[f'mfcc{i}_max'] = np.max(mfcc[i])
        w.writerow(row)